In [1]:
# Task 1: learn U3 parameters so a parameterized ansatz equals Toffoli
import pennylane as qml
from pennylane import numpy as np
np.set_printoptions(precision=6, suppress=True)

In [9]:
# --- Build Toffoli target using pennylane.numpy for autodiff compatibility ---
def toffoli_matrix():
    T = np.eye(8, dtype=complex)
    # swap |110> (6) <-> |111> (7)
    T = T.copy()
    T[6,6] = 0.0
    T[7,7] = 0.0
    T[6,7] = 1.0
    T[7,6] = 1.0
    return np.array(T, dtype=complex)  # this is pennylane.numpy array

TOFF = toffoli_matrix()  # shape (8,8), dtype=complex

In [10]:
# --- U3 helper (RZ RY RZ) ---
def U3(params, wires):
    theta, phi, lam = params
    qml.RZ(phi, wires=wires)
    qml.RY(theta, wires=wires)
    qml.RZ(lam, wires=wires)

In [11]:
# ansatz topology (replaceable with the exact PDF topology)
def ansatz_circuit(params):
    # params shape: (L, 3, 3) -> L layers, 3 qubits, 3 params per U3
    for layer in range(params.shape[0]):
        for w in range(3):
            U3(params[layer, w], wires=w)
        # simple entangling pattern (adjust if you want)
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])

In [12]:
# device and QNode returning state; use adjoint diff for correctness+speed
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev, diff_method="adjoint", interface="autograd")
def ansatz_state(params, input_index):
    # prepare computational basis state |input_index> on wires [0,1,2]
    for i in range(3):
        if (input_index >> (2 - i)) & 1:  # msb is wire 0
            qml.PauliX(wires=i)
    ansatz_circuit(params)
    return qml.state()

In [16]:
# cost: 1 - average fidelity across basis states, compute inner product with qml.math.vdot
def cost(params):
    total = 0.0
    for i in range(8):
        sv = ansatz_state(params, i)         # returns pennylane numpy complex vector
        target = TOFF[:, i]                  # column i of target
        # Use qml.math.vdot for autodiff-friendly inner product:
        ov = qml.math.vdot(target, sv)       # complex scalar (autodiff-friendly)
        fid = qml.math.real(qml.math.conj(ov) * ov) # squared overlap
        total = total + fid
    avg_fid = total / 8.0
    return 1.0 - avg_fid

In [17]:
# --- Optimization ---
def make_init_params(L):
    return np.random.randn(L, 3, 3) * 0.1

L = 2
params = make_init_params(L)
opt = qml.AdamOptimizer(stepsize=0.1)

print("Starting optimization...")
for step in range(200):
    params = opt.step(cost, params)
    if step % 20 == 0 or step == 199:
        c = cost(params)
        print(f"Step {step:03d} cost={c:.6e}")
        if c < 1e-6:
            break

Starting optimization...


C:\Users\SUJAN\.conda\envs\main\Lib\site-packages\autograd\numpy\numpy_wrapper.py:187: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


Step 000 cost=2.594304e-01
Step 020 cost=5.509694e-01
Step 040 cost=3.173524e-01
Step 060 cost=4.399408e-01
Step 080 cost=4.759766e-01
Step 100 cost=4.270636e-01
Step 120 cost=4.142226e-01
Step 140 cost=4.794003e-01
Step 160 cost=4.466539e-01
Step 180 cost=4.707141e-01
Step 199 cost=4.513481e-01


In [18]:
print("Final cost:", cost(params))
# print found parameters
for l in range(L):
    for q in range(3):
        print(f"Layer {l}, wire {q}, U3 params = {params[l,q]}")


Final cost: 0.4513481123346398
Layer 0, wire 0, U3 params = [ 0.918676 -0.522882 -1.679788]
Layer 0, wire 1, U3 params = [0.054735 0.580163 5.953855]
Layer 0, wire 2, U3 params = [ 1.570781  3.147688 -0.000024]
Layer 1, wire 0, U3 params = [ 0.68332  -1.810864  0.648251]
Layer 1, wire 1, U3 params = [-0.1474    5.969653  3.383275]
Layer 1, wire 2, U3 params = [ 1.570692 -0.000035 -3.078942]
